In [1]:
import time
import numpy as np
import pygame

from NovelSwarmBehavior.novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from NovelSwarmBehavior.novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from NovelSwarmBehavior.novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from NovelSwarmBehavior.novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from NovelSwarmBehavior.novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from NovelSwarmBehavior.novel_swarms.behavior.SensorOffset import GeneElementDifference
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.novelty.BehaviorDiscovery import BehaviorDiscovery
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.sensors.BinaryLOSSensor import BinaryLOSSensor
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from data.swarmset import SwarmDataset, DataBuilder
from generation.halted_evolution import HaltedEvolution

def build_random_dataset():
    sensors = SensorSet([
        GenomeBinarySensor(genome_id=8),
        GenomeBinarySensor(genome_id=9)
    ])

    agent_config = DiffDriveAgentConfig(
        sensors=sensors,
        seed=None,
    )

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.15, round_digits=4, exclude=[(-0.15, 0.15)]),
        GeneRule(_max=((1/3) * np.pi), _min=-((2/3) * np.pi), mutation_step=(np.pi/8), round_digits=4),
        GeneRule(_max=((2/3) * np.pi), _min=-((1/3) * np.pi), mutation_step=(np.pi/8), round_digits=4),
    ]

    phenotype = [
        AverageSpeedBehavior(),
        AngularMomentumBehavior(),
        RadialVarianceBehavior(),
        ScatterBehavior(),
        GroupRotationBehavior(),
        GeneElementDifference(8, 9)
    ]

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=24,
        seed=None,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=100,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=600,
        display_novelty=True,
        save_archive=True,
        show_gui=True,
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = HaltedEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    baseline_data = DataBuilder("data/full-dual-sensors", steps=1000, agents=24, ev=halted_evolution, screen=screen)
    baseline_data.create()
    baseline_data.evolution.close()

build_random_dataset()

pygame 2.1.2 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
0.0% Complete
0.02% Complete
0.04% Complete
0.06% Complete
0.08% Complete
0.1% Complete
0.12% Complete
0.14% Complete
0.16% Complete
0.18% Complete
0.2% Complete
0.22% Complete
0.24% Complete
0.26% Complete
0.28% Complete
0.3% Complete
0.32% Complete
0.34% Complete
0.36% Complete
0.38% Complete
0.4% Complete
0.42% Complete
0.44% Complete
0.46% Complete
0.48% Complete
0.5% Complete
0.52% Complete
0.54% Complete
0.56% Complete
0.58% Complete
0.6% Complete
0.62% Complete
0.64% Complete
0.66% Complete
0.68% Complete
0.7% Complete
0.72% Complete
0.74% Complete
0.76% Complete
0.78% Complete
0.8% Complete
0.82% Complete
0.84% Complete
0.86% Complete
0.88% Complete
0.9% Complete
0.92% Complete
0.94% Complete
0.96% Complete
0.98% Complete
1.0% Complete
1.02% Complete
1.04% Complete
1.06% Complete
1.08% Complete
1.1% Complete
1.12% Complete
1.14% Complete
1.16% Complete
1.18% Complete

# Embed into Latent Space

In [31]:
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
from scipy import ndimage
import random

def batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.batch_network_from_numpy(anchor_images, pos_images, neg_images)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image, with_transforms=False):
    optim.zero_grad()
    if with_transforms:
        anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    else:
        anchor_out, pos_out, neg_out = network.network_from_numpy(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def pretraining(data, network, loss_fn, optim, data_cutoff=None, data_size=500):
    if data_cutoff is None:
        data_cutoff = len(data) - 1
    samples = np.random.random_integers(0, data_cutoff, (data_size, 2))
    total_loss = 0
    total_updates = 0
    for i, s in enumerate(samples):
        if i % (data_size // 20) == 0:
            print(f"Unsupervised Training.. {(i * 100) / data_size}")
        anchor_image = data[s[0]][0]
        pos_image = data[s[0]][0]

        pos_images = np.stack([
            [ndimage.rotate(pos_image, 90)],
            [ndimage.rotate(pos_image, 180)],
            [ndimage.rotate(pos_image, 270)],
            # ndimage.gaussian_filter(pos_image, 1),
            # ndimage.gaussian_filter(pos_image, 3),
            # ndimage.gaussian_filter(pos_image, 5),
        ])

        anchor_images = np.stack([[anchor_image] for _ in pos_images])
        neg_images = np.stack([ [data[random.randint(0, data_cutoff)][0]] for _ in pos_images])

        loss = batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images)
        total_loss += loss.item()
        total_updates += 1

    return total_loss / total_updates


def human_in_the_loop(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at):
    print("HIL TIME!")
    improvements, human_loss, triplet_helpfulness, embedded_archive = hil.humanInput(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at)
    print(f"Improvement Count: {improvements}, loss: {human_loss}")

    HIL_archive.save_to_file(f"data/queries/{trial_name}_hil.csv")
    random_archive.save_to_file(f"data/queries/{trial_name}_rand.csv")
    return improvements, human_loss, triplet_helpfulness, embedded_archive

In [14]:
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL

import numpy as np
import time

from torch.utils.tensorboard import SummaryWriter

trial_name = f"{str(int(time.time()))}"

TRAIN = False
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network = NoveltyEmbedding().to(device)

SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = True
EVOLUTION = True
PASS_THROUGHS = 1
DATA_SIZE = 5000
EVOLUTIONS_PER_EPOCH = 12
K_CLUSTERS = 8

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-dual-sensors", rank=0)
# evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=800, n_agents=24, evolve_population=100, seed=None)
evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=1200, n_agents=24, evolve_population=80, seed=None)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.8)

# Margin was 10
loss_fn = torch.nn.TripletMarginLoss(margin=15)
hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()
random_archive = DataAggregationArchive(scalar=True)
EPSILON = 0.5

simulation_time = 0
evolution_time = 0
training_time = 0
hil_time = 0

# TODO: Add Randomly-sampled Contrastive/Triplet Loss
if PRETRAINING:
    network.load_model("unsupervised_decay_0.7_target_0.08")
    print("Pretrained model loaded!")

dataset = anchor_dataset if EVOLUTION else sampled_dataset

if TRAIN:
    writer = SummaryWriter()
    STOP_FLAG = False
    for epoch in range(PASS_THROUGHS):

        if EVOLUTION:
            start_time = time.time()
            for gen in range(EVOLUTIONS_PER_EPOCH if epoch > 0 else 3):
                # Simulate current population + Save Data
                for i in range(len(evolution.getPopulation())):
                    # The collection of the original behavior vector below is only used to collect data to compare with the baseline
                    visual_behavior, genome, baseline_behavior = evolution.next()
                    dataset.new_entry(visual_behavior, genome, baseline_behavior)
                simulation_time += (time.time() - start_time)

                # Then, evolve
                start_time = time.time()
                embedded_archive = hil.getEmbeddedArchive(dataset, network)
                evolution.overwriteArchive(embedded_archive, random_archive)
                embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
                evolution.overwriteBehavior(embedded_behavior)
                evolution.evolve()
                evolution.restart_screen()
                evolution_time += (time.time() - start_time)

                print(f"Evolution complete for e{epoch} and gen{gen}")

                # Record the accuracy of the medoids with respect to the synthetic policy
                medoid_acc, cluster_acc = 0, 0
                if len(dataset) > 0 and SAVE_CLUSTER_MEDOIDS:
                    if SYNTHETIC_HIL and EVOLUTION:
                        hil.synthetic_knowledge = hil.syntheticBehaviorSpace(dataset)
                        print(f"Synthetic Human Knowledge Size: {len(hil.synthetic_knowledge.labels_)}")
                    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

                # Cluster current dataset, display clusters, and save for analysis
                if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
                    hil.embed_and_cluster(network, dataset, auto_quit=True)
                    evolution.restart_screen()

        # Human in the Loop determines behavior embedding
        if HUMAN_IN_LOOP:
            start_time = time.time()
            improvements, human_loss, triplet_helpfulness, _ =   human_in_the_loop(
                                                                dataset,
                                                                network,
                                                                optimizer,
                                                                loss_fn,
                                                                HIL_archive,
                                                                random_archive,
                                                                len(dataset)
                                                            )
            hil_time += (time.time() - start_time)

        # Train on past user information
        if TRAIN:
            start_time = time.time()
            loss = None

            average_loss = 50
            loop = 0
            TRANSFORMS = 20
            network.train()
            while (average_loss > 0.01) and loop < 20:

                loss_sum = 0
                total_loss = 0

                # Shuffled Batching
                samples = [l for batch in]

                for i, (anchor, pos, neg) in enumerate(HIL_archive):
                    anchor_encoding = dataset[anchor][0]
                    similar_encoding = dataset[pos][0]
                    anti_encoding = dataset[neg][0]
                    optimizer.zero_grad()

                    anchor_out, pos_out, neg_out = network.network_from_numpy(anchor_encoding, similar_encoding, anti_encoding)
                    loss = loss_fn(anchor_out, pos_out, neg_out)
                    loss_sum += loss.item()
                    if loss.item() > 0:
                        loss.backward()
                        optimizer.step()
                    total_loss += 1

                    # anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_encoding, similar_encoding, anti_encoding)
                    # loss = loss_fn(anchor_out, pos_out, neg_out)
                    # loss_sum += loss.item()
                    # if loss.item() > 0:
                    #     loss.backward()
                    #     optimizer.step()
                    # total_loss += 1

                    if loss is not None and i > 0 and i % 20 == 0:
                        print(f"Epoch Progress: {(i*100) / len(HIL_archive)}%, Immediate Loss: {loss.item()}")

                average_loss = loss_sum / (total_loss + 1)
                print(f"Loop: {loop}, Average Loss: {average_loss}")
                loop += 1

            print(f"Epoch: {epoch}")
            training_time += (time.time() - start_time)

            writer.add_scalar("Loss/Average", loss_sum / (total_loss + 1), epoch)

        if SAVE_CLUSTER_MEDOIDS:
            writer.add_scalar("Accuracy/MedoidClassification", medoid_acc, epoch)
            writer.add_scalar("Accuracy/RandomSampleClassification", cluster_acc, epoch)

        if HUMAN_IN_LOOP:
            writer.add_scalar("Loss/TripletQuality", triplet_helpfulness, epoch)
            writer.add_scalar("Queries/Total_Human_Queries", epoch*8*9, epoch)
            writer.add_scalar("Queries/Total_Triplets_Generated", len(HIL_archive), epoch)
            writer.add_scalar("Queries/Total_Random_Classes", len(random_archive), epoch)

        if EVOLUTION:
            writer.add_scalar("Novelty/Highest", evolution.behavior_discovery.getBestScore(), epoch)
            writer.add_scalar("Novelty/Average", evolution.behavior_discovery.getAverageScore(), epoch)

        writer.add_scalar("Time/Simulation", simulation_time, epoch)
        writer.add_scalar("Time/HIL", hil_time, epoch)
        writer.add_scalar("Time/Evolution", evolution_time, epoch)
        writer.add_scalar("Time/Training", training_time, epoch)

        scheduler.step()

evolution.close()

HIL Init!
Pretrained model loaded!


In [3]:
import pygame
pygame.quit()

# Pretraining

In [37]:
# Save Model
target = 0.08
loss = 50
# network = NoveltyEmbedding(out_size=15).to(device)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)
if PRETRAINING:
    epochs = 0
    while loss > target:
        loss = pretraining(sampled_dataset, network, loss_fn, optimizer, data_cutoff=None, data_size=500)
        print(f"Epoch {epochs}, loss: {loss}")
        epochs += 1
        scheduler.step()

network.save_model()

/tmp/ipykernel_1529200/1632704364.py:32: DeprecationWarning: This function is deprecated. Please call randint(0, 4999 + 1) instead
  samples = np.random.random_integers(0, data_cutoff, (data_size, 2))


Unsupervised Training.. 0.0
Unsupervised Training.. 5.0
Unsupervised Training.. 10.0
Unsupervised Training.. 15.0
Unsupervised Training.. 20.0
Unsupervised Training.. 25.0
Unsupervised Training.. 30.0
Unsupervised Training.. 35.0
Unsupervised Training.. 40.0
Unsupervised Training.. 45.0
Unsupervised Training.. 50.0
Unsupervised Training.. 55.0
Unsupervised Training.. 60.0
Unsupervised Training.. 65.0
Unsupervised Training.. 70.0
Unsupervised Training.. 75.0
Unsupervised Training.. 80.0
Unsupervised Training.. 85.0
Unsupervised Training.. 90.0
Unsupervised Training.. 95.0
Epoch 0, loss: 0.26957821917533875
Unsupervised Training.. 0.0
Unsupervised Training.. 5.0
Unsupervised Training.. 10.0
Unsupervised Training.. 15.0
Unsupervised Training.. 20.0
Unsupervised Training.. 25.0
Unsupervised Training.. 30.0
Unsupervised Training.. 35.0
Unsupervised Training.. 40.0
Unsupervised Training.. 45.0
Unsupervised Training.. 50.0
Unsupervised Training.. 55.0
Unsupervised Training.. 60.0
Unsupervised

KeyboardInterrupt: 

In [36]:
network.save_model()

In [ ]:
print(optimizer.lr)

In [ ]:
import time
import torch
from data.swarmset import SwarmDataset, DataBuilder
from networks.embedding import NoveltyEmbedding
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.results.results import main as results
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from data.swarmset import SwarmDataset, DataBuilder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding().to(device)
network.load_model("trialA-10-18-2022")
# anchor_dataset = SwarmDataset("data/full", rank=0)

WRITE_OUT = True
if WRITE_OUT:
    network.eval()
    test_archive = NoveltyArchive()
    for i in range(len(anchor_dataset)):
        anchor_encoding, genome, _ = anchor_dataset[i]
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        test_archive.addToArchive(embedding, genome)

In [ ]:
from ui.clustering_gui import ClusteringGUI
import pygame

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=test_archive, k_clusters=6, world_config=world_config, tsne_perplexity=16, tsne_early_exaggeration=12, skip_tsne=False)
gui = ClusteringGUI(config)
gui.displayGUI()
# results(config)
pygame.quit()

# Clustering + Analysis

In [ ]:
from ui.clustering_gui import ClusteringGUI

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(human_l_hist, "b", label='Human Loss')
plt.ylabel("Loss")
plt.xlabel("Time (Epochs)")
plt.legend()